# Авиарейсы без потерь   

Специализация: Data Science  
Группа: DST-29  
Выполнила: Марина Розова 

In [186]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Загрузим наш датасет

In [187]:
df = pd.read_csv('fly_data.csv')

In [188]:
display(df.head(5))
df.info() 
df.columns

,fl_id,dep_airport,arr_airport,craft_model,fly_time,pass_count,fly_revenue
0,136119,AAQ,SVO,Boeing 737-300,101.0,113.0,1653000.0
1,136120,AAQ,SVO,Boeing 737-300,99.0,109.0,1605400.0
2,136122,AAQ,SVO,Boeing 737-300,100.0,97.0,1431000.0
3,136130,AAQ,SVO,Boeing 737-300,99.0,107.0,1556600.0
4,136131,AAQ,SVO,Boeing 737-300,99.0,124.0,1812800.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fl_id        193 non-null    int64  
 1   dep_airport  193 non-null    object 
 2   arr_airport  193 non-null    object 
 3   craft_model  193 non-null    object 
 4   fly_time     193 non-null    float64
 5   pass_count   180 non-null    float64
 6   fly_revenue  180 non-null    float64
dtypes: float64(3), int64(1), object(3)
memory usage: 10.7+ KB


Index(['fl_id', 'dep_airport', 'arr_airport', 'craft_model', 'fly_time',
       'pass_count', 'fly_revenue'],
      dtype='object')

**Описание признаков:**  
fl_id - идентификатор рейса  
dep_airport - аэропорт отправления  
arr_airport - аэропорт прибытия  
craft_model - модель самолёта  
fly_time - длительность полёта в минутах  
pass_count - количество пассажиров на борту  
fly_revenue - сумма, полученная за авиарейс  

Посмотрим на пропуски

In [189]:
# определение количества пропусков в датасете
def missing_values_table(df):
    # Всего отсутствующих значений
    mis_val = df.isnull().sum()

    # Процент пропущенных значений
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Составим таблицу с результатами
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Переименуем столбцы
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Пропуски', 1: '% пропусков'})

   # Отсортируем таблицу по проценту пропущенных по убыванию
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% пропусков', ascending=False).round(1)

    # Выведем таблицу на экран
    print("В нашем датафрейме " + str(df.shape[1]) + " столбцов.\n"
          "В " + str(mis_val_table_ren_columns.shape[0]) +
          " колонках есть пропущенные данные.")

    # Вернуть фрейм данных с отсутствующей информацией
    return mis_val_table_ren_columns

missing_values_table(df)

В нашем датафрейме 7 столбцов.
В 2 колонках есть пропущенные данные.


,Пропуски,% пропусков
pass_count,13,6.7
fly_revenue,13,6.7


Скорее всего эти пропуски связаны со слотом, купленным авиакомпанией.  
[Пустые рейсы. Почему самолеты иногда летают без пассажиров?](https://34travel.me/post/ghost-flights)

Эти данные бесполезны для нашего анализа, поэтому можем их удалить

In [190]:
df = df.dropna()

Для решения нашей задачи рассчитаем прибыльность рейса, исходя из следующей формулы:  

**Прибыльность рейса = Сумма за авиарейс - Стоимость топлива на перелет - З/п экипажа - Эксплуатационные расходы**

**Зарплата экипажа** - величина фиксированная и составляет 20% от суммы за авиаперелет при максимальной загрузке рейса.

**Затраты на топливо.**
Примем некоторые допущения по затратам топлива для самолета. Помимо топлива на рейс требуется некий запас на определенные цели - рулежка (примерно 7%), топливо если самолет не сможет сесть в запланированном аэропорту (примерно 19%), топливо для нахождения в зоне ожидания (33%), топливо на непредвиденные обстоятельства (6%). Совокупная надбавка к топливу на полет составит ориентировочно 65%.  

**Эксплуатационные расходы** - это затраты на обслуживания парка (16%), налоги и сборы государства (14%), техобслуживание (11%). Зафиксируем эуту величину в размере 41% от максимальной стоимости рейса - при учете полной загрузки самолета.

[Билет на самолет: сколько получает авиакомпания?](https://www.avianews.com/airlines/advice/2012/21_07_airline_costs.shtml)

 # Приступим к расчетам.  
   
 Рассчитаем стоимость топлива.  
 [ФЕДЕРАЛЬНОЕ АГЕНТСТВО ВОЗДУШНОГО ТРАНСПОРТА](https://favt.gov.ru/stat-date-gsm-price/?id=7329![image.png](attachment:image.png))

In [191]:
df_fuel = pd.read_csv('price_fuel.csv', sep=';')

In [192]:
display(df_fuel.head())

,date,price_fuel
0,2017-12-01,47101.0
1,2017-01-01,41435.0
2,2017-02-01,39553.0


In [193]:
df_fuel['price_fuel'] = pd.to_numeric(df_fuel['price_fuel'])

In [194]:
df_fuel.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        3 non-null      object 
 1   price_fuel  3 non-null      float64
dtypes: float64(1), object(1)
memory usage: 176.0+ bytes


 Расчет стоимости будем вести на основе среднего значения зимнего периода и добавлением НДС 18%.

In [195]:
price_fuel = round((df_fuel.price_fuel.sum()/df_fuel.price_fuel.count())*1.18,2)

In [196]:
price_fuel

50381.67

**50381.67** - средняя цена топлива за 1 тонну с учетом НДС 18%

**Загрузим данные по самолетам.**

[Boeing 737-300](https://www.airlines-inform.ru/commercial-aircraft/boeing-737-300.html)  
[SuperJet 100](https://www.airlines-inform.ru/commercial-aircraft/superjet-100.html)  
[SuperJet 100](https://ru.wikipedia.org/wiki/Sukhoi_Superjet_100#Лётно-технические_характеристики)  
[SuperJet 100](http://superjet.wikidot.com/wiki:fuel-real-flight)  

In [197]:
df_air = df_fuel = pd.read_csv('aircraft characteristics.csv', sep=';')

In [198]:
df_air

,craft_model,max_score,max_pass_seat,fuel_consumption
0,Boeing 737-300,795,130,2600
1,Sukhoi Superjet-100,870,97,1969


In [199]:
df_air.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   craft_model       2 non-null      object
 1   max_score         2 non-null      int64 
 2   max_pass_seat     2 non-null      int64 
 3   fuel_consumption  2 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 192.0+ bytes


Добавим данные о характеристиках самолета в датасет

In [200]:
df = df.merge(df_air, on='craft_model')
df

,fl_id,dep_airport,arr_airport,craft_model,fly_time,pass_count,fly_revenue,max_score,max_pass_seat,fuel_consumption
0,136119,AAQ,SVO,Boeing 737-300,101.0,113.0,1653000.0,795,130,2600
1,136120,AAQ,SVO,Boeing 737-300,99.0,109.0,1605400.0,795,130,2600
2,136122,AAQ,SVO,Boeing 737-300,100.0,97.0,1431000.0,795,130,2600
3,136130,AAQ,SVO,Boeing 737-300,99.0,107.0,1556600.0,795,130,2600
4,136131,AAQ,SVO,Boeing 737-300,99.0,124.0,1812800.0,795,130,2600
...,...,...,...,...,...,...,...,...,...,...
175,136943,AAQ,EGO,Sukhoi Superjet-100,50.0,94.0,733800.0,870,97,1969
176,136951,AAQ,EGO,Sukhoi Superjet-100,49.0,90.0,720600.0,870,97,1969
177,136953,AAQ,EGO,Sukhoi Superjet-100,50.0,97.0,765300.0,870,97,1969
178,136956,AAQ,EGO,Sukhoi Superjet-100,50.0,96.0,746400.0,870,97,1969


Добавим данные с затратами на топлива в наш датасет.

**Затраты на топливо = Время полета в часах  * Расход топлива (кг/ч) * Надбавка к топливу (в %) * Цена топлива за 1 кг** 

In [201]:
df['expense_fuel'] = round((((df['fly_time']/60)*df['fuel_consumption'])*1.65)*price_fuel/1000,2)

In [202]:
df

,fl_id,dep_airport,arr_airport,craft_model,fly_time,pass_count,fly_revenue,max_score,max_pass_seat,fuel_consumption,expense_fuel
0,136119,AAQ,SVO,Boeing 737-300,101.0,113.0,1653000.0,795,130,2600,363831.23
1,136120,AAQ,SVO,Boeing 737-300,99.0,109.0,1605400.0,795,130,2600,356626.65
2,136122,AAQ,SVO,Boeing 737-300,100.0,97.0,1431000.0,795,130,2600,360228.94
3,136130,AAQ,SVO,Boeing 737-300,99.0,107.0,1556600.0,795,130,2600,356626.65
4,136131,AAQ,SVO,Boeing 737-300,99.0,124.0,1812800.0,795,130,2600,356626.65
...,...,...,...,...,...,...,...,...,...,...,...
175,136943,AAQ,EGO,Sukhoi Superjet-100,50.0,94.0,733800.0,870,97,1969,136402.07
176,136951,AAQ,EGO,Sukhoi Superjet-100,49.0,90.0,720600.0,870,97,1969,133674.03
177,136953,AAQ,EGO,Sukhoi Superjet-100,50.0,97.0,765300.0,870,97,1969,136402.07
178,136956,AAQ,EGO,Sukhoi Superjet-100,50.0,96.0,746400.0,870,97,1969,136402.07


**Затраты на экипаж**

In [203]:
#Максимальная выручка Boeing 737-300
max_fly_revenue_Boeing = df[(df['pass_count']==130) & (df['craft_model']=='Boeing 737-300')]['fly_revenue'].max()

#Максимальная выручка Sukhoi Superjet-100
max_fly_revenue_Superjet = df[(df['pass_count']==97) & (df['craft_model']=='Sukhoi Superjet-100')]['fly_revenue'].max()

In [204]:
#Зарплата экипажа Boeing 737-300
salary_expenses_Boeing = max_fly_revenue_Boeing*0.2

#Зарплата экипажа Sukhoi Superjet-100
salary_expenses_Superjet = max_fly_revenue_Superjet*0.2

print('Зарплата экипажа:')
print(f'\tBoeing 737-300: {int(salary_expenses_Boeing)} рублей')
print(f'\tSukhoi Superjet-100: {int(salary_expenses_Superjet)} рублей')

Зарплата экипажа:
	Boeing 737-300: 377200 рублей
	Sukhoi Superjet-100: 153060 рублей


**Эксплуатационные расходы**

In [205]:
#Зарплата экипажа Boeing 737-300
technical_expenses_Boeing = max_fly_revenue_Boeing*0.41

#Зарплата экипажа Sukhoi Superjet-100
technical_expenses_Superjet = max_fly_revenue_Superjet*0.41

print('Эксплуатационные расходы:')
print(f'\tBoeing 737-300: {int(technical_expenses_Boeing)} рублей')
print(f'\tSukhoi Superjet-100: {int(technical_expenses_Superjet)} рублей')

Эксплуатационные расходы:
	Boeing 737-300: 773260 рублей
	Sukhoi Superjet-100: 313773 рублей


**Прибыльности рейсов**

In [206]:
def expenses(model):
    expense = salary_expenses_Boeing + technical_expenses_Boeing
    if model == 'Sukhoi Superjet-100':
        expense = salary_expenses_Superjet + technical_expenses_Superjet
    return expense

In [207]:
df['fly_expenses'] = df['craft_model'].apply(expenses)

In [208]:
df['profit'] = df['fly_revenue'] - df['expense_fuel'] - df['fly_expenses']

In [209]:
df

,fl_id,dep_airport,arr_airport,craft_model,fly_time,pass_count,fly_revenue,max_score,max_pass_seat,fuel_consumption,expense_fuel,fly_expenses,profit
0,136119,AAQ,SVO,Boeing 737-300,101.0,113.0,1653000.0,795,130,2600,363831.23,1150460.0,138708.77
1,136120,AAQ,SVO,Boeing 737-300,99.0,109.0,1605400.0,795,130,2600,356626.65,1150460.0,98313.35
2,136122,AAQ,SVO,Boeing 737-300,100.0,97.0,1431000.0,795,130,2600,360228.94,1150460.0,-79688.94
3,136130,AAQ,SVO,Boeing 737-300,99.0,107.0,1556600.0,795,130,2600,356626.65,1150460.0,49513.35
4,136131,AAQ,SVO,Boeing 737-300,99.0,124.0,1812800.0,795,130,2600,356626.65,1150460.0,305713.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,136943,AAQ,EGO,Sukhoi Superjet-100,50.0,94.0,733800.0,870,97,1969,136402.07,466833.0,130564.93
176,136951,AAQ,EGO,Sukhoi Superjet-100,49.0,90.0,720600.0,870,97,1969,133674.03,466833.0,120092.97
177,136953,AAQ,EGO,Sukhoi Superjet-100,50.0,97.0,765300.0,870,97,1969,136402.07,466833.0,162064.93
178,136956,AAQ,EGO,Sukhoi Superjet-100,50.0,96.0,746400.0,870,97,1969,136402.07,466833.0,143164.93


In [210]:
print(f'Количество убыточных зимних рейсов составляет {df[df.profit<0].shape[0]} шт.')
df[df['profit']<0]

Количество убыточных зимних рейсов составляет 20 шт.


,fl_id,dep_airport,arr_airport,craft_model,fly_time,pass_count,fly_revenue,max_score,max_pass_seat,fuel_consumption,expense_fuel,fly_expenses,profit
2,136122,AAQ,SVO,Boeing 737-300,100.0,97.0,1431000.0,795,130,2600,360228.94,1150460.0,-79688.94
14,136178,AAQ,SVO,Boeing 737-300,99.0,99.0,1434600.0,795,130,2600,356626.65,1150460.0,-72486.65
16,136185,AAQ,SVO,Boeing 737-300,103.0,104.0,1520000.0,795,130,2600,371035.81,1150460.0,-1495.81
18,136202,AAQ,SVO,Boeing 737-300,102.0,100.0,1495600.0,795,130,2600,367433.52,1150460.0,-22293.52
19,136204,AAQ,SVO,Boeing 737-300,100.0,107.0,1457800.0,795,130,2600,360228.94,1150460.0,-52888.94
28,136250,AAQ,SVO,Boeing 737-300,101.0,99.0,1407800.0,795,130,2600,363831.23,1150460.0,-106491.23
30,136253,AAQ,SVO,Boeing 737-300,98.0,93.0,1407800.0,795,130,2600,353024.36,1150460.0,-95684.36
44,136306,AAQ,SVO,Boeing 737-300,100.0,102.0,1443200.0,795,130,2600,360228.94,1150460.0,-67488.94
56,136352,AAQ,SVO,Boeing 737-300,100.0,92.0,1372400.0,795,130,2600,360228.94,1150460.0,-138288.94
57,136360,AAQ,SVO,Boeing 737-300,100.0,97.0,1455400.0,795,130,2600,360228.94,1150460.0,-55288.94


**Точка безубыточности**

In [211]:
df['general_expenses']=df['expense_fuel']+df['fly_expenses']

In [212]:
SS = "Sukhoi Superjet-100"
fly_time_SS = 50 # minutes
max_pass_count_SS = 97 # max passengers count
max_revenue_SS = df.query('craft_model==@SS'
                    and 'pass_count==max_pass_count'
                    and 'fly_time==@fly_time_SS').fly_revenue.max()
rev_pass_SS = max_revenue_SS / max_pass_count_SS # Средний доход с 1 пассажира.
idx = df.query('craft_model==@SU'
                    and 'pass_count==max_pass_count'
                    and 'fly_time==@fl_time_SU').fly_revenue.idxmax()
# Расходы (плановое время полёта, максимальное количество пассажиров):
expense_SS = df.loc[idx, 'general_expenses']


profit_zero_SS = expense_SS * max_pass_count_SS / max_revenue_SS

In [213]:
BG = "Boeing 737-300"
fly_time_BG = 100 # minutes
max_pass_count_BG = 130 # max passengers count
max_revenue_BG = df.query('craft_model==@BG'
                    and 'pass_count==max_pass_count'
                    and 'fly_time==@fl_time_BG').fly_revenue.max()
rev_pass_BG = max_revenue_BG / max_pass_count_BG # Средний доход с 1 пассажира.
idx = df.query('craft_model==@BG'
                    and 'pass_count==max_pass_count'
                    and 'fly_time==@fl_time_BG').fly_revenue.idxmax()
# Расходы (плановое время полёта, максимальное количество пассажиров):
expense_BG = df.loc[idx, 'general_expenses']


profit_zero_BG = expense_BG * max_pass_count_BG / max_revenue_BG

In [219]:
profit_zero_SS = expense_SS * max_pass_count_SS / max_revenue_SS
profit_zero_SS = int(round(profit_zero_SS, 0))

print('Точка безубыточности:')
print(f'\tSukhoi Superjet-100: {profit_zero_SS} пассажиров')


Точка безубыточности:
	Sukhoi Superjet-100: 76 пассажиров


In [220]:
profit_zero_BG = expense_BG * max_pass_count_BG / max_revenue_BG
profit_zero_BG = int(round(profit_zero_BG, 0))
print('Точка безубыточности:')
print(f'\tBoeing 737-300: {profit_zero_BG} пассажиров')

Точка безубыточности:
	Boeing 737-300: 104 пассажиров


In [225]:
profit_mean = round(df[df.profit>=0].profit.mean(), 0)
profit_max = round(df[df.profit>=0].profit.max(), 0)
print(f'Средняя прибыль оставшихся рейсов: {int(profit_mean)} рублей')
print(f'Максимальная прибыль рейса: {int(profit_max)} рублей')

Средняя прибыль оставшихся рейсов: 145542 рублей
Максимальная прибыль рейса: 382516 рублей
